In [34]:
import bs4 as bs
import urllib.parse
import urllib.request
import pandas as pd
from datetime import datetime

fname = 'Steam_Sales_'+datetime.now().strftime("%d_%m_%Y")+'.xlsx'
page = 1
delimiter = '€'
data = []
title = []

#Looping through the pages and collecting the data.
while page < 40:
    
    url = 'https://store.steampowered.com/search/?category1=998&specials=1'
    params = {'page':page}
    
    url_parts = list(urllib.parse.urlparse(url))
    query = dict(urllib.parse.parse_qsl(url_parts[4]))
    query.update(params)
    url_parts[4] = urllib.parse.urlencode(query)
    parsed = urllib.parse.urlunparse(url_parts)

    source = urllib.request.urlopen(parsed)
    soup = bs.BeautifulSoup(source,'lxml')

    for div in soup.find_all('span', {'class':'title'}):
        title.append(div.text)
        
    for div in soup.find_all('div', {'class':'col search_price_discount_combined responsive_secondrow'}):
        data.append(div.text.replace('\n', ' ').replace('%', '€').replace('-', ' ').replace(' ', '').
                    replace(',', '.').split(delimiter))  
    page += 1
    
#Creating the DataFrame.
df = pd.DataFrame(data, columns =['Discount', 'Original price', 'Discounted price', 'Title'])
df['Title'] = title
df['Discount'] = df['Discount'].astype(int).div(100)

# Creating a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(fname, engine='xlsxwriter', options={'strings_to_numbers': True})

# Converting the dataframe to an XlsxWriter Excel object.
df.to_excel(writer,sheet_name='Test', index=False)

#Creating cell formats using the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Test']
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format': '0%'})

#Applying formats to selected columns
worksheet.set_column('A:A', 10, format2)
worksheet.set_column('B:B', 12, format1)
worksheet.set_column('C:C', 15, format1)
worksheet.set_column('D:D', 40, None)

# Closing the Pandas Excel writer to output the Excel file.
writer.save()